In [2]:
import time
import requests
from tqdm import tqdm_notebook
from bs4 import BeautifulSoup

# Как устроен веб?

## Клиент-серверная архитектура

Компьютеры, подключенные к сети называются клиентами и серверами. Схема обмена информацией выглядит приблизительно так:
![image.png](https://mdn.mozillademos.org/files/8973/Client-server.jpg)

Клиенты являются обычными пользователями, подключенными к Интернету посредством устройств и программного обеспечения, доступного на этих устройствах (как правило, браузер, например, Firefox или Chrome).

Серверы - компьютеры, которые хранят веб-страницы, сайты или приложения. Когда клиентское устройство пытается получить доступ к веб-странице, копия страницы загружается с сервера на клиентский компьютер для отображения в браузере пользователя.

## HTTP как основной протокол передачи данных

Связь между клиентом и сервером осуществляется по определённому протоколу, обычно это HTTP (HyperText Transfer Protocol). Каждое HTTP-сообщение состоит из трёх частей, которые передаются в указанном порядке:

* Стартовая строка (Starting line) — определяет тип сообщения;
* Заголовки (Headers) — характеризуют тело сообщения, параметры передачи и прочие сведения;
* Тело сообщения (Message Body) — непосредственно данные сообщения. Обязательно должно отделяться от заголовков пустой строкой.

Тело сообщения может отсутствовать, но стартовая строка и заголовок являются обязательными элементами.

### Стартовая строка

Стартовые строки различаются для запроса и ответа. 

**Строка запроса** выглядит так: **`Метод URI HTTP/Версия`**. 

Самые часто используемые методы это `GET` и `POST` (ещё существую `DELETE`, `PUT`, `HEAD` и другие).

`GET` используется для запроса содержимого указанного ресурса, `POST` применяется для передачи пользовательских данных заданному ресурсу. 

**Cтрока ответа** сервера имеет следующий формат: **`HTTP/Версия КодСостояния Пояснение`**.

Код состояния является частью первой строки ответа сервера. Он представляет собой целое число из трёх цифр. Первая цифра указывает на класс состояния. За кодом ответа обычно следует отделённая пробелом поясняющая фраза на английском языке, которая разъясняет человеку причину именно такого ответа. [Список кодов состояния на wiki](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D0%BE%D0%B4%D0%BE%D0%B2_%D1%81%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D1%8F_HTTP).

# Модуль Requests

Библиотека requests — это обёртка над другой, более низкоуровневой библиотекой urllib3, упрощающая доступ ко многим функциям.

В requests имеется:

* Множество методов http аутентификации
* Сессии с куками
* Полноценная поддержка SSL
* Различные методы .json(), которые вернут данные в нужном формате
* Проксирование
* Грамотная и логичная работа с исключениями

Вот как выглядит HTTP-запрос методом `GET` с помощью requests:

In [1]:
import requests
r = requests.get("https://www.hse.ru/")
print(f"Код состояния: {r.status_code}.")
print(f"Заголовки: {r.headers['content-type']}.")

Код состояния: 200.
Заголовки: text/html; charset=utf-8.


Получить HTML можно с помощью свойства `text`:

In [12]:
r.text

'<!DOCTYPE html>\n<html>\n\n\n\n\n\n\n\n\n\n\n<head>\n\t<title>Национальный исследовательский университет Высшая школа экономики</title>\n\t<meta charset="utf-8" />\n\t<meta name="viewport" content="width=device-width, initial-scale=1">\n\t<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\n\t<meta name="yandex-verification" content="25e3420f8bfc397e" />\n\t<meta name="theme-color" content="#1658DA"/>\n\t<link rel="manifest" href="https://www.hse.ru/f/src/manifest/manifest_ru.json">\n\t<link rel="stylesheet" href="/f/src/global/css/main_fonts.css" />\n\t<link rel="stylesheet" href="/f/src/global/css/main_icons.css" />\n\t<link rel="stylesheet" href="/f/src/global/css/main_forms.css" />\n\t<link rel="stylesheet" href="/f/src/global/css/fotorama.css" />\n\t<link rel="stylesheet" href="/f/src/global/css/sitemap.css" />\n\t<link rel="stylesheet" href="/f/src/global/css/main_hse.css" />\n\t<link rel="stylesheet" href="/f/src/home/main_en.css" />\n\t<link rel="stylesheet" href="/f/src/h

# HTML и его парсинг с помощью BeautifulSoup

HTML — теговый язык разметки документов. Любой документ на языке HTML представляет собой набор элементов, причём начало и конец каждого элемента обозначается специальными пометками — тегами. Элементы могут быть пустыми, то есть не содержащими никакого текста и других данных (например, тег перевода строки `<br>`). В этом случае обычно не указывается закрывающий тег. Кроме того, элементы могут иметь атрибуты, определяющие какие-либо их свойства (например, размер шрифта для тега <font>). Атрибуты указываются в открывающем теге. Вот примеры фрагментов HTML-документа:

даст следующее:

> <strong>Текст между двумя тегами — открывающим и закрывающим.</strong>

> <a href="http://www.example.com">Здесь элемент содержит атрибут href, то есть гиперссылку.</a>

> А вот пример пустого элемента: <br> и какой-то текст

Регистр, в котором набрано имя элемента и имена атрибутов, в HTML значения не имеет (в отличие от XHTML). Элементы могут быть вложенными. 

Для парсинга HTML существуют разные библиотеки, но чаще всего используется [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). Для тех, кто занком с jQuery более удобным вариантом может быть библиотека [pyquery](https://pythonhosted.org/pyquery/).

In [ ]:
!pip3 install beautifulsoup4

In [13]:
from bs4 import BeautifulSoup

BS поддерживает разные парсеры html: https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser

In [32]:
soup = BeautifulSoup("какой-то текст <b class='class_name', id='id_attr'>Полужирный текст</b> <i>И ещё немножко</i>", "lxml")
soup

<html><body><p>какой-то текст <b class="class_name" id="id_attr">Полужирный текст</b> <i>И ещё немножко</i></p></body></html>

In [33]:
btag = soup.b
btag

<b class="class_name" id="id_attr">Полужирный текст</b>

In [34]:
dir(btag)

['HTML_FORMATTERS',
 'XML_FORMATTERS',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_attr_value_as_string',
 '_attribute_checker',
 '_find_all',
 '_find_one',
 '_formatter_for_name',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_select_debug',
 '_selector_combinators',
 '_should_pretty_print',
 '_tag_name_matches_and',
 'append',
 'attribselect_re',
 'attrs',
 'can_be_empty_element',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decomp

In [35]:
btag.name

'b'

In [36]:
btag.name = "span"
btag

<span class="class_name" id="id_attr">Полужирный текст</span>

In [37]:
btag["class"]

['class_name']

In [38]:
btag.attrs

{'class': ['class_name'], 'id': 'id_attr'}

In [39]:
btag["id"] = "some_id"
btag

<span class="class_name" id="some_id">Полужирный текст</span>

In [40]:
btag.string

'Полужирный текст'

In [41]:
btag.string.replace_with("Новый текст")
btag

<span class="class_name" id="some_id">Новый текст</span>

In [42]:
soup.get_text()

'какой-то текст Новый текст И ещё немножко'

In [45]:
sibling_soup = BeautifulSoup("<a><b>text1</b><c>text2</c></b></a>", "lxml")
print(sibling_soup.prettify())

<html>
 <body>
  <a>
   <b>
    text1
   </b>
   <c>
    text2
   </c>
  </a>
 </body>
</html>


# Сложности

Не всегда получить текст страницы бывает так просто, поскольку в современные веб-приложения загружают контент динамически, а URL при этом не изменяется (хотя должен бы). Для примера посмотрите на сайты https://www.1tv.ru/news и https://gorod55.ru/news. В таком случае при помощи Инструментов разработчика придётся отслеживать, какие запросы делает страница, и искать среди них те, которые возвращают нужную информацю.

# Самостоятельная работа

Скачайте профили всех сотрудников питерской вышки и определите, у кого из них больше всего публикаций. Переменные в помощь:

In [3]:
CYRILLIC_ALPHBET = "АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
HSE_URL = "https://www.hse.ru/org/persons/?ltr={};udept=135083"

In [4]:
responses = {}

start_time = time.time()

for letter in tqdm_notebook(CYRILLIC_ALPHBET):
    res = requests.get(HSE_URL.format(letter))
    if res.status_code == 200:
        responses[letter] = res.text
    else:
        raise Exception("Ошибка {} при загрузке по букве {}".format(res.status_code, letter))
        
end_time = time.time()

print("Время выполнения: {} секунд.".format(round(end_time - start_time, 2)))


Время выполнения: 69.27 секунд.


In [36]:
people_links = []
for letter, res in responses.items():
    soup = BeautifulSoup(res, "lxml")
    for person in soup.find("div", class_="persons__section").find_all("div", class_="person"):
        p_block = person.find(class_="link link_dark large b")
        people_links.append(("https://www.hse.ru" + p_block["href"], p_block.get_text().strip()))

In [ ]:
len(people_links) # ссылки на профили все сотрудников питерской вышки

Собираем все профили. Может занять много времени. Для этого и нужна асинхронность.

In [50]:
scoreboard = []
counter = 0
for url, name in people_links:
    res = requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, "lxml")
        pub_block = soup.find("div", class_="publications")
        if pub_block:
            pub_block.find_all("li")
            scoreboard.append((len(pub_block.find_all("li")), name, url))
        else:
            scoreboard.append((0, name, url))
    else:
        raise Exception("Ошибка {} при загрузке профиля {} ({})".format(res.status_code, url, name))
    counter += 1
    print(counter, end="\r")

In [51]:
sorted(scoreboard, key=lambda x: x[0], reverse=True)[:50]

[(111, 'Сунгуров Александр Юрьевич', 'https://www.hse.ru/org/persons/505427'),
 (87, 'Котляров Иван Дмитриевич', 'https://www.hse.ru/org/persons/26455742'),
 (74, 'Назарова Варвара Вадимовна', 'https://www.hse.ru/org/persons/26527205'),
 (73, 'Лимонов Леонид Эдуардович', 'https://www.hse.ru/org/persons/26661989'),
 (73, 'Макарова Василиса Александровна', 'https://www.hse.ru/staff/vmakarova'),
 (69, 'Николенко Сергей Игоревич', 'https://www.hse.ru/org/persons/56987224'),
 (69,
  'Омельченко Елена Леонидовна',
  'https://www.hse.ru/org/persons/14509451'),
 (66,
  'Матвеенко Владимир Дмитриевич',
  'https://www.hse.ru/org/persons/202791'),
 (62, 'Аистов Андрей Валентинович', 'https://www.hse.ru/staff/aistov'),
 (59,
  'Казарцев Евгений Вячеславович',
  'https://www.hse.ru/org/persons/106252141'),
 (56, 'Гордин Валерий Эрнстович', 'https://www.hse.ru/staff/gordin'),
 (56,
  'Заостровцев Андрей Павлович',
  'https://www.hse.ru/org/persons/14037002'),
 (56, 'Селин Адриан Александрович', 'htt

# Асинхронные запросы

Синхронные операции — операции, при которых мы получаем результат в результате блокирования потока выполнения.

Асинхронные операции — операции, при которых мы просим совершить некоторую операцию и можем каким-либо образом отслеживать процесс/результат её выполнения. Когда она будет выполнена — неизвестно, но мы можем продолжить заниматься другими делами.

Написание асинхронного кода [не самая простая задача для разработчика](http://lucumr.pocoo.org/2016/10/30/i-dont-understand-asyncio/), но именно асинхронность может повысить производительность вашей программы в несколько десятков раз.

Ключевое слово [**`await`**](https://docs.python.org/3.6/reference/expressions.html#await) указывает, что при выполнении следующего за ним выражения возможно переключение с текущей сопрограммы на другую или на основной поток выполнения.
Соответственно выражение после await тоже не простое, это должен быть [**`awaitable`**](https://docs.python.org/3.6/glossary.html#term-awaitable) объект. Используя await в какой-либо корутине, мы таким образом объявляем, что корутина может отдавать управление обратно в event loop, который, в свою очередь, запустит какую-либо следующую задачу.

**async**

* **`async def`** — определяет native coroutine function, результатом вызова которой будет объект-сопрограмма native coroutine, пока еще не запущенная.

* **`async for`** — определяет, что итератор используемый в цикле, при получении следующего значения может переключать выполнение с текущей сопрограммы.

* **`async with`** — определяет, что при входе в контекстный блок и выходе из него может быть переключение выполнения с текущей сопрограммы.

asyncio оперирует следующими понятиями: циклы событий, корутины и футуры.

* **цикл событий (event loop)** управляет выполнением различных задач: регистрирует поступление и запускает в подходящий момент. Это бесконечный цикл, который берёт события из очереди и как-то их обрабатывает. А в некоторых промежутках — смотрит, не произошло ли каких-нибудь IO-событий и тогда добавляет в очередь событие об этом, чтобы потом обработать.
* **корутины** — специальные функции, похожие на генераторы python, от которых ожидают (await), что они будут отдавать управление обратно в цикл событий. Необходимо, чтобы они были запущены именно через цикл событий
* **футуры** — объекты, в которых хранится текущий результат выполнения какой-либо задачи. Это может быть информация о том, что задача ещё не обработана или уже полученный результат; а может быть вообще исключение

In [19]:
async def get_persons(url, letters, campus_id, simultaneous_requests_num=100):
    tasks = []
    # создаём объект класса Semaphore с ограничением в 100 корутин
    sem = asyncio.Semaphore(simultaneous_requests_num)

    # Создаём клиентскую сессию, в контексте которой будут выполняться все запросы
    async with ClientSession() as session:
        for letter in letters:
            url_params = {"letter": letter, "campus_id": campus_id}
            url = url.format(**url_params)
            # Функция ensure_future принимает на вход корутину и планирует её выполнение,
            # возврящая future вида Task. Таким образом корутина помещается в цикл событий.
            # https://docs.python.org/3/library/asyncio-task.html#asyncio.Task
            task = asyncio.ensure_future(
                fetch_url(session, sem, url=url, key=letter))
            tasks.append(task)
        # собираем все futures в один awaitable список
        responses = await asyncio.gather(*tasks)
        return responses


# создаём цикл событий
loop = asyncio.get_event_loop()

# планируем выполнение новой корутины
uber_future = asyncio.ensure_future(
    get_persons(HSE_URL, letters=CYRILLIC_ALPHBET, campus_id=135083))
# запускаем запланированные задачи и получаем список результатов.
persons_by_letter = loop.run_until_complete(uber_future)

In [41]:
async def get_profiles(urls, simultaneous_requests_num=100):
    tasks = []
    sem = asyncio.Semaphore(simultaneous_requests_num)

    async with ClientSession() as session:
        for url in urls:
            task = asyncio.ensure_future(fetch_url(session, sem, url=url))
            tasks.append(task)
        responses = await asyncio.gather(*tasks)
        return responses

In [42]:
# создаём цикл событий
loop = asyncio.get_event_loop()

# планируем выполнение новой корутины
uber_future = asyncio.ensure_future(get_profiles((url for name, url in persons_w_urls)))
# запускаем запланированные задачи и получаем список результатов.
profiles = loop.run_until_complete(uber_future)

Допишите код, чтобы скачать все профили.